# 2nd Level Model Structure: Reboiler

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.energy_reboiler import energy_block_rule
from physics.VLE_reboiler_MPCC_P import VLE_block_rule
from physics.MPCC_P import P_NCP_block_rule, P_Reg_block_rule, P_pf_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,350+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.Q_main = pe.Var(within=pe.Reals) # MW

# Construct Individual Blocks

In [5]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [6]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.P_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [7]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [8]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

# MPCC - P-Pspec

In [9]:
model.MPCC = pe.Block(rule = P_pf_block_rule)

> Importing MPCC_P_Reg Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.s_L
| MPCC.s_V
| MPCC.pf
| MPCC.rho
--------------------------------------------------
> Spliting pressure used in VLE



## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [10]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [11]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [12]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load a flash liquid composition as feed

In [13]:
with open('../saved_solutions/reactive_flash_MPCC_P_pf_200C.pickle', 'rb') as f:
    C200 = pickle.load(f)

## Fixing In/Product Redundent Flow

In [14]:
model.y_.fix(0)
for i in m.COMP_TOTAL:
    model.x_['in',i].fix(C200.Solution.Variable['x[{}]'.format(i)]['Value'])
model.L['in'].fix(1)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_V_.fix(0)
model.H_L_.fix(C200.Solution.Variable['H_L']['Value'])

## Fixing Model Parameters

In [15]:
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(0)
model.z.fix(0)
model.VLE_block.n_ave.fix(20)
model.Q_main.fix(0)

In [16]:
check_DOF(pe,model)

Active Equality Constraints:	 957
Active Inequality Constraints:	 1
Active Variables:		 1132
Fixed Variables:		 172
DOF:				 3


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [17]:
model.T.setub(300+273.15)

In [18]:
# model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)
# model.obj = pe.Objective(expr = model.MPCC.pf,sense=pe.minimize)
model.obj = pe.Objective(expr = - model.MPCC.pf ,sense=pe.maximize)

In [19]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

  82  3.2062398e+00 1.24e+02 1.70e+04  -1.0 4.36e+03    -  1.00e-01 3.81e-03w  1
  83  5.4226766e+00 4.44e-01 1.68e+03  -1.0 2.52e+03    -  1.02e-01 3.15e-03h  7
  84  5.4076243e+00 4.50e-01 1.84e+03  -1.0 7.06e+03    -  2.98e-01 3.15e-03h  8
  85  5.3923604e+00 4.57e-01 1.91e+03  -1.0 7.03e+03    -  1.24e-01 3.16e-03h  8
  86  5.3772072e+00 4.63e-01 2.82e+03  -1.0 7.01e+03    -  8.95e-01 3.16e-03h  8
  87  5.3616442e+00 4.70e-01 3.43e+03  -1.0 6.97e+03    -  1.55e-01 3.18e-03h  8
  88  5.3462639e+00 4.76e-01 8.31e+03  -1.0 6.96e+03    -  1.00e+00 3.18e-03h  8
  89  5.3306029e+00 4.83e-01 1.08e+04  -1.0 6.92e+03    -  2.15e-01 3.19e-03h  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90  5.3150365e+00 4.89e-01 2.58e+04  -1.0 6.90e+03    -  1.00e+00 3.19e-03h  8
  91  5.2992402e+00 4.95e-01 3.93e+04  -1.0 6.86e+03    -  3.79e-01 3.20e-03h  8
  92  5.2834174e+00 5.02e-01 7.21e+04  -1.0 6.84e+03    -  6.07e-01 3.21e-03h  8
  93  3.2367375e+00 1.27e+02

 176  4.3634481e+00 8.15e-01 1.03e+12  -1.0 5.71e+03    -  1.72e-01 3.53e-03h  8
 177  4.3500374e+00 8.19e-01 1.09e+12  -1.0 5.69e+03    -  1.00e+00 3.54e-03h  8
 178  4.3366583e+00 8.23e-01 1.10e+12  -1.0 5.67e+03    -  1.82e-01 3.54e-03h  8
 179  4.3233012e+00 8.27e-01 1.16e+12  -1.0 5.65e+03    -  1.00e+00 3.55e-03h  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180  4.3100019e+00 8.31e-01 1.16e+12  -1.0 5.63e+03    -  1.87e-01 3.56e-03h  8
 181  4.2967239e+00 8.35e-01 1.22e+12  -1.0 5.61e+03    -  1.00e+00 3.56e-03h  8
 182  4.2834828e+00 8.39e-01 1.22e+12  -1.0 5.60e+03    -  1.96e-01 3.57e-03h  8
 183  4.2702612e+00 8.43e-01 1.28e+12  -1.0 5.58e+03    -  1.00e+00 3.57e-03h  8
 184  2.5816698e+00 1.12e+02 7.39e+11  -1.0 5.56e+03    -  2.07e-01 4.58e-01w  1
 185  2.4604432e+00 1.07e+02 6.56e+11  -1.0 2.99e+03    -  1.68e-02 4.27e-02w  1
 186  2.4415496e+00 1.07e+02 6.09e+12  -1.0 1.83e+03    -  3.95e-02 2.03e-03w  1
 187  4.2570691e+00 8.47e-01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 280  3.5917617e+00 9.83e-01 2.18e+12  -1.0 4.50e+03    -  1.00e+00 3.06e-04h 10
 281  3.5899145e+00 9.83e-01 2.18e+12  -1.0 4.50e+03    -  1.29e-02 5.93e-04h  9
 282  3.5881604e+00 9.82e-01 2.18e+12  -1.0 4.49e+03    -  2.44e-01 5.62e-04h  9
 283  3.5611690e+00 1.00e+00 2.17e+12  -1.0 4.48e+03    -  1.59e-02 8.63e-03h  5
 284  3.5455581e+00 1.01e+00 2.19e+12  -1.0 4.45e+03    -  1.00e+00 4.92e-03h  6
 285  3.5410800e+00 1.00e+00 2.19e+12  -1.0 4.45e+03    -  3.76e-02 1.46e-03h  8
 286  3.5364089e+00 1.00e+00 2.19e+12  -1.0 4.45e+03    -  2.23e-01 1.52e-03h  8
 287  3.5315491e+00 1.00e+00 2.19e+12  -1.0 4.45e+03    -  5.74e-02 1.59e-03h  8
 288  2.8841722e+00 1.71e+01 2.15e+12  -1.0 4.44e+03    -  1.00e+00 2.12e-01w  1
 289  2.7779344e+00 1.69e+01 2.05e+12  -1.0 3.56e+03    -  2.59e-02 4.36e-02w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290  2.7764042e+00 1.68e+01

 377  3.1297888e+00 9.98e-01 3.13e+12  -1.0 3.90e+03    -  8.66e-02 2.20e-03h  8
 378  3.1237400e+00 9.97e-01 3.38e+12  -1.0 3.89e+03    -  5.70e-01 2.27e-03h  8
 379  2.3280716e+00 2.63e+01 2.26e+12  -1.0 3.88e+03    -  1.08e-01 2.99e-01w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380  2.1216595e+00 2.53e+01 2.16e+12  -1.0 2.79e+03    -  1.00e+00 1.10e-01w  1
 381  2.0633439e+00 2.46e+01 3.33e+12  -1.0 2.46e+03    -  1.00e+00 3.17e-02w  1
 382  3.1175239e+00 9.97e-01 3.38e+12  -1.0 2.44e+03    -  1.08e-01 2.34e-03h  7
 383  3.1047421e+00 9.98e-01 3.38e+12  -1.0 3.88e+03    -  1.00e+00 4.82e-03h  7
 384  3.0912854e+00 1.00e+00 3.39e+12  -1.0 3.86e+03    -  1.17e-01 5.10e-03h  7
 385  3.0771727e+00 1.00e+00 3.39e+12  -1.0 3.84e+03    -  7.05e-01 5.38e-03h  7
 386  3.0624325e+00 1.01e+00 3.39e+12  -1.0 3.83e+03    -  1.56e-01 5.66e-03h  7
 387  3.0470662e+00 1.01e+00 3.39e+12  -1.0 3.81e+03    -  1.00e+00 5.93e-03h  7
 388  3.0311118e+00 1.01e+00

 484  3.5857146e-01 2.71e-03 5.84e+09  -1.0 8.58e-02    -  9.90e-01 5.00e-01f  2
 485  4.0058274e-01 2.59e-07 1.85e+06  -1.0 4.47e-02    -  9.91e-01 1.00e+00f  1
 486  4.0055119e-01 2.33e-07 1.73e+05  -1.0 3.97e-03    -  1.00e+00 1.03e-01f  2
 487  3.9994700e-01 4.77e-11 5.76e+00  -1.0 6.04e-04    -  1.00e+00 1.00e+00f  1
 488  3.5136864e-05 4.09e-11 4.07e+02  -5.7 4.00e-01    -  1.00e+00 1.00e+00f  1
 489  6.9947627e-06 2.69e-10 2.98e-05  -5.7 3.21e-05    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490  5.5696546e-06 2.70e-10 7.37e-05  -5.7 2.34e-04    -  1.00e+00 1.00e+00h  1
 491  5.5464793e-06 1.20e-10 4.27e-07  -5.7 6.12e-05    -  1.00e+00 1.00e+00h  1
 492  2.1605125e-08 1.21e-10 1.73e+00  -8.6 5.58e-06    -  9.99e-01 9.89e-01h  1
 493 -8.4482047e-09 7.28e-12 2.06e-08  -8.6 1.10e-06    -  1.00e+00 1.00e+00f  1
 494 -9.2759200e-09 1.35e-10 1.69e-07  -9.0 1.70e-06    -  1.00e+00 1.00e+00h  1
 495 -8.8950885e-09 1.35e-10

In [20]:
model.Q_main.value

0

In [21]:
model.T.value

473.15

In [24]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reboiler_MPCC_P_pf_200C.pickle','wb') as f:
#     pickle.dump(results,f)

In [23]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

Component		Liquid: 1.0000			Vapor: 0.0000			log K
------------------------------------------------------------------------------------------------------------
H2         		 0.8837%			25.2934%			1.4567
CO         		 1.3401%			28.6944%			1.3307
CO2        		 3.7245%			31.8624%			0.9322
H2O        		 3.6023%			12.4034%			0.5370
C2H4       		 0.0174%			 0.1174%			0.8302
C3H6       		 0.0444%			 0.1351%			0.4830
C4H8       		 0.0456%			 0.1204%			0.4221
C5H10      		 0.0722%			 0.0931%			0.1104
C6H12      		 0.1262%			 0.0865%			-0.1641
C7H14      		 0.2078%			 0.0799%			-0.4151
C8H16      		 0.3264%			 0.0731%			-0.6499
C9H18      		 0.4920%			 0.0659%			-0.8729
C10H20     		 0.7120%			 0.0583%			-1.0869
C11H22     		 0.6286%			 0.0320%			-1.2938
C12H24     		 0.8302%			 0.0265%			-1.4952
C13H26     		 1.0411%			 0.0212%			-1.6918
C14H28     		 1.2375%			 0.0161%			-1.8847
C15H30     		 1.3970%			 0.0118%			-2.0743
C16H32     		 1.5062%			 0.0083%			-2.2611
C17H34     		 1.5634%			 0.0056%

# Iterative Solve for Data Analysis

In [ ]:
update_dual(pe,model)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
rf_data = {}
rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = [];
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = [];
    
Trange = np.arange(350+273.15,199+273.15,-5)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {:.2f} K\t|\tV = {:.5f}\t|\tL = {:.5f}\t|'.format(model.T.value,model.V['out'].value,model.L['out'].value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)

# VLE Validation using AspenPlus

### Feed (Reactor Effluent)

In [ ]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.T.value-273.15))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.L['out'].value+model.V['out'].value))
print('-'*72)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.x[i].value*model.L['out'].value\
        +model.y[i].value*model.V['out'].value)/(model.L['out'].value+model.V['out'].value)))

### Phase Separation

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

In [ ]:
list(zip(rf_data['T'],rf_data['Q']))